In [10]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [2]:
import mne

In [28]:
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian

In [3]:
raw = mne.io.read_raw_edf('data/Ears_lr1.edf', preload=True)

Extracting EDF parameters from /home/delta_c/EnW/Biotech_Researches/EEG_Event_Classification/data/Ears_lr1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 70999  =      0.000 ...   141.998 secs...


/tmp/ipykernel_4953/995447494.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'SAO2'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf('data/Ears_lr1.edf', preload=True)


In [29]:
sample_rate = raw.info['sfreq']

# Searching events

In [4]:
stamps = pd.read_csv('data/click_timestamps1.csv')
stamps.head()

,Ear,Time
0,Left,2024-10-09 16:02:40.026203
1,Right,2024-10-09 16:02:42.027171
2,Left,2024-10-09 16:02:44.027530
3,Right,2024-10-09 16:02:46.027743
4,Left,2024-10-09 16:02:48.027924


In [23]:
left_stamps = [datetime.strptime(stamp, '%Y-%m-%d %H:%M:%S.%f') for stamp in stamps[stamps['Ear'] == 'Left']['Time']]
right_stamps = [datetime.strptime(stamp, '%Y-%m-%d %H:%M:%S.%f') for stamp in stamps[stamps['Ear'] == 'Right']['Time']]
right_stamps[:2]

[datetime.datetime(2024, 10, 9, 16, 2, 42, 27171),
 datetime.datetime(2024, 10, 9, 16, 2, 46, 27743)]

In [26]:
initial_stamp = datetime.strptime(datetime.strftime(raw.info['meas_date'], '%Y-%m-%d %H:%M:%S.%f'), '%Y-%m-%d %H:%M:%S.%f')
initial_stamp

datetime.datetime(2024, 10, 9, 16, 2, 35)

In [32]:
events = {
    'left': [round(sample_rate * (left_stamps[i] - initial_stamp).total_seconds()) for i in range(len(left_stamps))],
    'right': [round(sample_rate * (right_stamps[i] - initial_stamp).total_seconds()) for i in range(len(right_stamps))]
}
events['right']

[3514,
 5514,
 7514,
 9514,
 11514,
 13515,
 15515,
 17515,
 19515,
 21516,
 23516,
 25516,
 27516,
 29516,
 31517,
 33517,
 35517,
 37517,
 39518,
 41518,
 43518,
 45518,
 47518,
 49519,
 51519,
 53519,
 55519,
 57519,
 59520,
 61520,
 63520,
 65520,
 67520]

# Fetching windows